In [ ]:
import glob
import os
import warnings

import geopandas
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors
import pandas
import seaborn

from cartopy import crs as ccrs
from mpl_toolkits.axes_grid1 import make_axes_locatable

In [ ]:
# from geopandas/geoseries.py:358, when using geopandas.clip:
#
# UserWarning: GeoSeries.notna() previously returned False for both missing (None) and empty geometries.
# Now, it only returns False for missing values. Since the calling GeoSeries contains empty geometries, 
# the result has changed compared to previous versions of GeoPandas.
#
# Given a GeoSeries 's', you can use '~s.is_empty & s.notna()' to get back the old behaviour.
#
# To further ignore this warning, you can do: 
warnings.filterwarnings('ignore', 'GeoSeries.notna', UserWarning)

In [ ]:
# default to larger figures
plt.rcParams['figure.figsize'] = 10, 10

# Postprocessing and plotting EEH analysis
Scenarios
- [x] Colour coded map showing the percentage changes in EEH population by LAD 
- [x] Total EEH population compared with ONS projection 
- [x] Total housing growth per LAD, 2015-2020, 2020-2030, 2030-2040, 2040-2050 (may be better as cumulative chart with LADs)

Pathways 
- [x] Proportion of engine types for each Pathway 2015-2050 
- [x] Annual CO2 emission * 5 Pathways 2015, 2020, 2030, 2040, 2050 
- [x] Colour coded map showing Vehicle km in 2050 for each LAD * 5 Pathways
- [x] Annual electricity consumption for car trips * 5 Pathways, 2015, 2020, 2030, 2040, 2050 
- [x] Congestion/capacity utilisation in 2050 for each LAD * 5 Pathways (map/chart)


In [ ]:
all_zones = geopandas.read_file('../preparation/Local_Authority_Districts__December_2019__Boundaries_UK_BUC-shp/Local_Authority_Districts__December_2019__Boundaries_UK_BUC.shp')

In [ ]:
zone_codes = pandas.read_csv('lads-codes-eeh.csv').lad19cd

In [ ]:
eeh_zones = all_zones \
    [all_zones.lad19cd.isin(zone_codes)] \
    [['lad19cd', 'lad19nm', 'st_areasha', 'geometry']]
eeh_zones.plot()

In [ ]:
scenarios = [os.path.basename(d) for d in sorted(glob.glob('eeh/0*'))]
scenarios

In [ ]:
timesteps = [os.path.basename(d) for d in sorted(glob.glob('eeh/01-BaU/*'))]
timesteps

## Population scenario

In [ ]:
def read_pop(fname):
    pop = pandas.read_csv(fname)
    pop = pop \
        [pop.year.isin([2015, 2050])] \
        .melt(id_vars='year', var_name='lad19cd', value_name='population')    
    pop = pop[pop.lad19cd.isin(zone_codes)] \
        .pivot(index='lad19cd', columns='year')
    pop.columns = ['pop2015', 'pop2050']

    pop['perc_change'] = (pop.pop2050 - pop.pop2015) / pop.pop2015
    pop.perc_change *= 100
    return pop

eehpop = read_pop('../preparation/data/csvfiles/eehPopulation.csv')
arcpop = read_pop('../preparation/data/csvfiles/eehArcPopulationBaseline.csv')

In [ ]:
eehpop.sort_values(by='perc_change').tail()

In [ ]:
def plot_pop(eeh_zones, pop):
    df = eeh_zones.merge(pop, on='lad19cd', validate='one_to_one')
    
    fig, ax = plt.subplots(1, 1)
    ax.xaxis.set_visible(False)
    ax.yaxis.set_visible(False)

    divider = make_axes_locatable(ax)

    cax = divider.append_axes("right", size="5%", pad=0.1)
    
    df.plot(column='perc_change', ax=ax, legend=True, cax=cax, cmap='coolwarm_r', vmax=95, vmin=-95)

    cax.yaxis.set_label_text('Population (% change 2015-2050)')
    cax.yaxis.get_label().set_visible(True)

    return fig

In [ ]:
eehpop.to_csv('eehPopulationChange.csv')

In [ ]:
fig = plot_pop(eeh_zones, eehpop)
plt.savefig("eehPopulationChange.png")
plt.savefig("eehPopulationChange.svg")

In [ ]:
fig = plot_pop(eeh_zones, arcpop)
plt.savefig("snppPopulationChange.png")
plt.savefig("snppPopulationChange.svg")

## Results

In [ ]:
def read_result(fname, scenarios, timesteps):
    dfs = []
    for s in scenarios:
        for t in timesteps:
            path = os.path.join('eeh', s, t, fname)
            _, ext = os.path.splitext(fname)
            if ext == '.csv':
                df = pandas.read_csv(path)
            elif ext in ('.shp', '.gpkg', '.geojson'):
                df = geopandas.read_file(path)
            else:
                raise Exception(f"Don't know how to read files of type '{ext}'")
            df['year'] = t
            df['scenario'] = s
            dfs.append(df)
    return pandas.concat(dfs)

## CO2 Emissions

In [ ]:
zone_vehicle_emissions = read_result('totalCO2EmissionsZonalPerVehicleType.csv', scenarios, timesteps)
zone_vehicle_emissions.head(2)

In [ ]:
annual_eeh_emissions = zone_vehicle_emissions[zone_vehicle_emissions.zone.isin(zone_codes)] \
    .groupby(['scenario', 'year']) \
    .sum()
annual_eeh_emissions['TOTAL'] = annual_eeh_emissions.sum(axis=1)
annual_eeh_emissions.to_csv('eehCO2Emissions.csv')
annual_eeh_emissions.head(10)

## Vehicle km per LAD

In [ ]:
vkm_a = read_result('vehicleKilometresWithAccessEgress.csv', scenarios, timesteps)
eeh_vkm_a = vkm_a[vkm_a.zone.isin(zone_codes)] \
    .set_index(['scenario', 'year', 'zone'])
eeh_vkm_a['TOTAL'] = eeh_vkm_a.sum(axis=1)
eeh_vkm_a.to_csv('eehVehicleKilometresWithAccessEgress.csv')
eeh_vkm_a.head()

In [ ]:
vkm = read_result('vehicleKilometres.csv', scenarios, timesteps)
eeh_vkm = vkm[vkm.zone.isin(zone_codes)] \
    .set_index(['scenario', 'year', 'zone'])
eeh_vkm['TOTAL'] = eeh_vkm.sum(axis=1)
eeh_vkm.to_csv('eehVehicleKilometres.csv')
eeh_vkm.head()

In [ ]:
eeh_vkm.describe()

In [ ]:
df = eeh_vkm.reset_index().drop(columns='zone').groupby(['scenario', 'year']).sum()[['TOTAL']].reset_index()

seaborn.catplot(
    x = "year",
    y = "TOTAL",
    hue = "scenario",
    data = df,
    kind = "bar")

In [ ]:
def plot_vkm(eeh_zones, eeh_vkm, scenario, year):
    vmax = eeh_vkm.TOTAL.max()
    df = eeh_vkm[['TOTAL']].reset_index() \
        .rename(columns={'TOTAL': 'vkm'})
    df = df[(df.scenario == scenario) & (df.year == year)] \
        .drop(columns=['scenario', 'year'])
    df = geopandas.GeoDataFrame(df.merge(eeh_zones, left_on='zone', right_on='lad19cd', validate='one_to_one'))
    
    fig, ax = plt.subplots(1, 1)
    ax.xaxis.set_visible(False)
    ax.yaxis.set_visible(False)

    divider = make_axes_locatable(ax)

    cax = divider.append_axes("right", size="5%", pad=0.1)

    df.plot(column='vkm', ax=ax, legend=True, cax=cax, cmap='inferno', vmax=vmax)

    cax.yaxis.set_label_text('Vehicle kilometres (km)')
    cax.yaxis.get_label().set_visible(True)

    return fig

In [ ]:
fig = plot_vkm(eeh_zones, eeh_vkm, scenarios[0], "2015")
plt.savefig("eehVehicleKilometres2015.png")
plt.savefig("eehVehicleKilometres2015.svg")

for s in scenarios:
    fig = plot_vkm(eeh_zones, eeh_vkm, s, "2050")
    plt.savefig(f"eehVehicleKilometres2050_{s}.png")
    plt.savefig(f"eehVehicleKilometres2050_{s}.svg")

## Electricity consumption for car trips

In [ ]:
car_elec = read_result('zonalTemporalElectricityCAR.csv', scenarios, timesteps)
car_elec = car_elec[car_elec.zone.isin(zone_codes)] \
    .set_index(['scenario', 'year', 'zone'])
car_elec['TOTAL'] = car_elec.sum(axis=1)
car_elec.to_csv('eehZonalTemporalElectricityCAR.csv')
car_elec.head(2)

In [ ]:
car_energy = read_result('energyConsumptionsZonalCar.csv', scenarios, timesteps)
car_energy = car_energy[car_energy.zone.isin(zone_codes)] \
    .set_index(['scenario', 'year', 'zone'])
car_energy.to_csv('eehEnergyConsumptionsZonalCar.csv')
car_energy.head(2)

## Congestion/capacity utilisation

In [ ]:
zb = eeh_zones.bounds
extent = (zb.minx.min(), zb.maxx.max(), zb.miny.min(), zb.maxy.max())
extent

In [ ]:
network_base = read_result('outputNetwork.shp', [scenarios[0]], ["2015"])

In [ ]:
eeh_nb = network_base.cx[extent[0]:extent[1], extent[2]:extent[3]].copy()
eeh_nbc = geopandas.clip(eeh_nb, eeh_zones)

In [ ]:
eeh_nb.head(1)

In [ ]:
eeh_nb.drop(columns=['SRefE','SRefN','IsFerry', 'iDir', 'Anode', 'Bnode', 'CP', 'year', 'CapUtil', 'scenario']).to_file('eehNetwork.gpkg', driver='GPKG')

In [ ]:
def plot_cap(zones, network, network_clipped):
    fig, ax = plt.subplots(1, 1)
    ax.xaxis.set_visible(False)
    ax.yaxis.set_visible(False)

    divider = make_axes_locatable(ax)

    cax = divider.append_axes("right", size="5%", pad=0.1)

    zones.plot(ax=ax, color='#eeeeee', edgecolor='white')
    network.plot(ax=ax, color='#eeeeee')
    network_clipped.plot(column='CapUtil', ax=ax, legend=True, cax=cax, cmap='inferno', vmax=200)

    cax.yaxis.set_label_text('Capacity Utilisation (%)')
    cax.yaxis.get_label().set_visible(True)

    return fig

In [ ]:
fig = plot_cap(eeh_zones, eeh_nb, eeh_nbc)
plt.savefig('eehCapacity2015.png')
plt.savefig('eehCapacity2015.svg')

In [ ]:
for s in scenarios:
    network = read_result('outputNetwork.shp', [s], ["2050"])
    eeh_nb = network.cx[extent[0]:extent[1], extent[2]:extent[3]].copy()
    eeh_nbc = geopandas.clip(eeh_nb, eeh_zones)
    fig = plot_cap(eeh_zones, eeh_nb, eeh_nbc)
    plt.savefig(f'eehCapacity2050_{s}.png')
    plt.savefig(f'eehCapacity2050_{s}.svg')

In [ ]:
dfs = []

df = read_result('outputNetwork.shp', [scenarios[0]], ["2015"])

df = geopandas.clip(df, eeh_zones) \
    [['EdgeID', 'Anode', 'Bnode', 'CP', 'RoadNumber', 'iDir', 'SRefE',
       'SRefN', 'Distance', 'FFspeed', 'FFtime', 'IsFerry', 'Lanes', 'CapUtil',
       'year', 'scenario']]
dfs.append(df)

for s in scenarios:
    df = read_result('outputNetwork.shp', [s], ["2050"])
    df = geopandas.clip(df, eeh_zones) \
        [['EdgeID', 'Anode', 'Bnode', 'CP', 'RoadNumber', 'iDir', 'SRefE',
           'SRefN', 'Distance', 'FFspeed', 'FFtime', 'IsFerry', 'Lanes', 'CapUtil',
           'year', 'scenario']]
    dfs.append(df)
    
link_capacity = pandas.concat(dfs) \
    .set_index(['scenario', 'year'])
link_capacity.head(2)

In [ ]:
link_to_lad = geopandas.sjoin(eeh_nbc, eeh_zones, how="left", op='intersects') \
    [['EdgeID','lad19cd','lad19nm']] \
    .drop_duplicates(subset=['EdgeID'])
link_to_lad

In [ ]:
link_capacity

In [ ]:
link_capacity_with_lad = link_capacity \
    .reset_index() \
    .merge(link_to_lad, on='EdgeID', how='left') \
    .set_index(['scenario', 'year', 'EdgeID']) 

link_capacity_with_lad

In [ ]:
link_capacity_with_lad.to_csv('eehLinkCapUtil.csv')

In [ ]:
mean_cap = link_capacity_with_lad[['CapUtil', 'lad19cd','lad19nm']] \
    .reset_index() \
    .drop(columns='EdgeID') \
    .groupby(['scenario', 'year', 'lad19cd', 'lad19nm']) \
    .mean()
mean_cap.to_csv('eehLADAverageCapUtil.csv')
mean_cap

In [ ]:
df = mean_cap.reset_index()
print(len(df.scenario.unique()))
print(len(df.year.unique()))

print(len(df.lad19cd.unique()))
print(6 * 37)

## Link travel times/speeds

In [ ]:
link_times = read_result('linkTravelTimes.csv', scenarios, timesteps)
link_times.head(1)

In [ ]:
eeh_nbc

In [ ]:
eeh_lt = link_times[link_times.edgeID.isin(eeh_nbc.EdgeID)]

In [ ]:
eeh_lt.to_csv('eehLinkTravelTimes.csv', index=False)

In [ ]:
KM_TO_MILES = 0.6213712

In [ ]:
hours = [
    'MIDNIGHT', 'ONEAM', 'TWOAM', 'THREEAM', 'FOURAM', 'FIVEAM', 
    'SIXAM', 'SEVENAM', 'EIGHTAM', 'NINEAM', 'TENAM', 'ELEVENAM', 
    'NOON', 'ONEPM', 'TWOPM', 'THREEPM', 'FOURPM', 'FIVEPM', 
    'SIXPM', 'SEVENPM', 'EIGHTPM', 'NINEPM', 'TENPM', 'ELEVENPM'
]

In [ ]:
def merge_times_to_network(network_clipped, link_times, hours):
    # nbc is clipped network
    # lt is link times
    # hours is list of hour names
    
    # merge link times (by hour of day) onto network
    df = network_clipped \
        .drop(columns=['scenario', 'year']) \
        .rename(columns={'EdgeID': 'edgeID'}) \
        .merge(
            link_times,
            on="edgeID"
        ) \
        [[
            'edgeID', 'RoadNumber', 'iDir', 'Lanes', 'Distance', 'FFspeed',  
            'MIDNIGHT', 'ONEAM', 'TWOAM', 'THREEAM', 'FOURAM', 'FIVEAM', 
            'SIXAM', 'SEVENAM', 'EIGHTAM', 'NINEAM', 'TENAM', 'ELEVENAM', 
            'NOON', 'ONEPM', 'TWOPM', 'THREEPM', 'FOURPM', 'FIVEPM', 
            'SIXPM', 'SEVENPM', 'EIGHTPM', 'NINEPM', 'TENPM', 'ELEVENPM',
            'geometry'
        ]]
    # calculate flow speeds from distance / time * 60 [to get back to km/h] * 0.6213712 [to miles/h]
    for hour in hours:
        df[hour] = (df.Distance / df[hour]) * 60 * KM_TO_MILES
    df.FFspeed *= KM_TO_MILES
    return df

In [ ]:
eeh_ltb = merge_times_to_network(
    eeh_nbc, 
    eeh_lt[(eeh_lt.scenario == '01-BaU') & (eeh_lt.year == "2015")], 
    hours)
eeh_ltb

In [ ]:
eeh_ltb.columns

In [ ]:
def plot_speed(zones, network, network_clipped, col, label=None):
    fig, ax = plt.subplots(1, 1)
    ax.xaxis.set_visible(False)
    ax.yaxis.set_visible(False)

    divider = make_axes_locatable(ax)

    cax = divider.append_axes("right", size="5%", pad=0.1)

    zones.plot(ax=ax, color='#eeeeee', edgecolor='white')
    network.plot(ax=ax, color='#eeeeee')
    network_clipped.plot(column=col, ax=ax, legend=True, cax=cax, cmap='inferno', vmax=75, vmin=0)
    
    if label is not None:
        # place a text box in upper left in axes coords
        props = props = dict(boxstyle='round', facecolor='white', alpha=0.5)
        ax.text(0.05, 0.95, label, transform=ax.transAxes, fontsize=14,
            verticalalignment='top', bbox=props)

    cax.yaxis.set_label_text('Speed (km/h)')
    cax.yaxis.get_label().set_visible(True)

    return fig

In [ ]:
fig = plot_speed(eeh_zones, eeh_nb, eeh_ltb, 'EIGHTAM', "Morning peak")
fname = f"speed2015_peakam.png"
plt.savefig(fname)
plt.close(fig)

In [ ]:
fig = plot_speed(eeh_zones, eeh_nb, eeh_ltb, 'FFspeed', "Free flow")
fname = f"speed2015_free.png"
plt.savefig(fname)
plt.close(fig)

In [ ]:
for i, hour in enumerate(hours):
    fig = plot_speed(eeh_zones, eeh_nb, eeh_ltb, hour, f"{str(i).zfill(2)}:00")
    fname = f"speed2015_{str(i).zfill(3)}.png"
    print(fname, end=" ")
    plt.savefig(fname)
    plt.close(fig)

### Convert to GIF

Using imagemagick, needs installing, next line runs in the shell

In [ ]:
! convert -delay 20 -loop 0 speed2015_0*.png speed2015.gif

### Each scenario peak speeds in 2050

In [ ]:
for scenario in scenarios:
    ltb = merge_times_to_network(
        eeh_nbc, 
        eeh_lt[(eeh_lt.scenario == scenario) & (eeh_lt.year == "2050")], 
        hours)
    
    fig = plot_speed(eeh_zones, eeh_nb, ltb, 'EIGHTAM', "Morning peak")
    fname = f"speed2050_{scenario}_peakam.png"
    print(fname, end=" ")
    plt.savefig(fname)
    plt.close(fig)

## Rank links per-scenario for peak speed in 2050

In [ ]:
eeh_flow = eeh_lt[eeh_lt.year == "2050"] \
    [["scenario", "edgeID", "EIGHTAM", "freeFlow"]] \
    .rename(columns={'EIGHTAM': 'peakFlow'})

eeh_flow['flowRatio'] = eeh_flow.freeFlow / eeh_flow.peakFlow

eeh_flow.drop(columns=['peakFlow', 'freeFlow'], inplace=True)

eeh_flow = eeh_flow.pivot_table(columns='scenario', index='edgeID', values='flowRatio')
eeh_flow.columns.name = None
eeh_flow['bestScenarioAtPeak'] = eeh_flow.idxmax(axis=1)
eeh_flow.head(2)

In [ ]:
eeh_flow.groupby('bestScenarioAtPeak').count()[["01-BaU"]]

In [ ]:
eeh_flowg = eeh_nbc \
    [["EdgeID", "RoadNumber", "iDir", "Distance", "Lanes", "geometry"]] \
    .rename(columns={'EdgeID': 'edgeID'}) \
    .merge(
        eeh_flow,
        on="edgeID"
    )
lu = {
#     '01-BaU': '1:Business as Usual',
#     '02-HighlyConnected': '2:Highly Connected',
#     '03-AdaptedFleet': '3:Adapted Fleet',
#     '04-BehavShiftPolicy': '4:Behaviour Shift (policy-led)',
#     '05-BehavShiftResults': '5:Behaviour Shift (results-led)',
    '01-BaU': '01 BaU',
    '02-HighlyConnected': '02 HC',
    '03-AdaptedFleet': '03 AF',
    '04-BehavShiftPolicy': '04 BSp',
    '05-BehavShiftResults': '05 BSr',
}
eeh_flowg.bestScenarioAtPeak = eeh_flowg.bestScenarioAtPeak \
    .apply(lambda s: lu[s])
eeh_flowg.head(1)

In [ ]:
eehcm = matplotlib.colors.ListedColormap(
    [(74/255, 120/255, 199/255),
    (238/255, 131/255, 54/255),
    (170/255, 170/255, 170/255),
    (255/255, 196/255, 0/255),
    (84/255, 130/255, 53/255)],
    name='eeh')

In [ ]:
fig, ax = plt.subplots(1, 1)
ax.xaxis.set_visible(False)
ax.yaxis.set_visible(False)

eeh_zones.plot(ax=ax, color='#f2f2f2', edgecolor='white')
eeh_nb.plot(ax=ax, color='#eeeeee')
eeh_flowg.plot(column='bestScenarioAtPeak', ax=ax, legend=True, cmap=eehcm)
plt.savefig("bestScenarioPeakFlowRatio.png")
plt.savefig("bestScenarioPeakFlowRatio.svg")

## Link travel times direct